## Importing libraries

In [1]:
import pandas as pd
import numpy as np
import random

# For method 1
from random import randint
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

# For method 2
from numpy import dot
from numpy.linalg import norm 

# For method 3
from sklearn.metrics.pairwise import cosine_similarity
from surprise import SVD, Reader, Dataset, accuracy
from surprise.model_selection import train_test_split

# For method 4
from pyspark.ml.fpm import FPGrowth
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, LongType


# Generating data

In [2]:
def generate_data(n_books, n_genres, n_authors, n_publishers, n_readers, dataset_size):
    '''
    This function will generate a dataset with features associated to
    book data set. The dataset will have the following columns : 
        - book_id (String) : Unique identified for the book
        - book_rating (Integer) : A value between 0 and 10
        - reader_id (String) : Unique identifier for the user
        - book_genre (Integer) : An integer representing a genre for the book, 
                                 value is between 1 and 15, indicating that 
                                 there are 15 unique genres. Each book can only
                                 have 1 genre
        - author_id (String) : Unique identifier for the author of the book
        - num_pages (Integer) : Random value between 70 and 500
        - publisher_id (String) : A unique identifier for the publisher of the book
        - publish_year (Integer) : The year of book publishing
        - book_price (Integer) : The sale price of the book
        - text_lang (Integer) : The language of the book - returns an integer which 
                                is mapped to some language
        
    params:
        n_books (Integer) : The number of books you want the dataset to have
        n_genres (Integer) : Number of genres to be chosen from
        n_authors (Integer) : Number of authors to be generated
        n_publishers (Integer) : Number of publishers for the dataset
        n_readers (Integer) : Number of readers for the dataset
        dataset_size (Integer) : The number of rows to be generated 
        
    example:
        data = generate_data()
    '''
    
    d = pd.DataFrame(
        {
            'book_id' : [randint(1, n_books) for _ in range(dataset_size)],
            'author_id' : [randint(1, n_authors) for _ in range(dataset_size)],
            'book_genre' : [randint(1, n_genres) for _ in range(dataset_size)],
            'reader_id' : [randint(1, n_readers) for _ in range(dataset_size)],
            'num_pages' : [randint(75, 700) for _ in range(dataset_size)],
            'book_rating' : [randint(1, 10) for _ in range(dataset_size)],
            'publisher_id' : [randint(1, n_publishers) for _ in range(dataset_size)],
            'publish_year' : [randint(2000, 2021) for _ in range(dataset_size)],
            'book_price' : [randint(1, 200) for _ in range(dataset_size)],
            'text_lang' : [randint(1,7) for _ in range(dataset_size)]
        }
    ).drop_duplicates()
    return d

In [3]:
d = generate_data(n_books = 3000, n_genres = 10, n_authors = 450, n_publishers = 50, n_readers = 30000, dataset_size = 100000)
d.to_csv('./data/data.csv', index = False)

# Method 1. Collaborative Filtering System

**Intuition**

Collaborative filtering is the process of predicting the interests of a user by identifying preferences and information from many users. This is done by filtering data for information or patterns using techniques involving collaboration among multiple agents, data sources, etc. The underlying intuition behind collaborative filtering is that if user A and B have similar taste in a product, then A and B are likely to have similar taste in other products as well. 

There are two common types of approaches in collaborative filtering, memory based and model based approach.

**Memory based approaches** — also often referred to as neighbourhood collaborative filtering. Essentially, ratings of user-item combinations are predicted on the basis of their neighbourhoods. This can be further split into user based collaborative filtering and item based collaborative filtering. User based essentially means that likeminded users are going to yield strong and similar recommendations. Item based collaborative filtering recommends items based on the similarity between items calculated using user ratings of those items.

**Model based approaches** — are predictive models using machine learning. Features associated to the dataset are parameterized as inputs of the model to try to solve an optimization related problem. Model based approaches include using things like decision trees, rule based approaches, latent factor models etc.

**Advantages**

The main advantage to using collaborative filtering models is its simplicity to implement and the high level coverage they provide. It is also beneficial because it captures subtle characteristics (very true for latent factor models) and does not require understanding of the item content.

**Disadvantages**

The main disadvantage to this model is that it’s not friendly for recommending new items, this is because there has been no user/item interaction with it. This is referred to as the cold start problem. Memory based algorithms are known to perform poorly on highly sparse datasets.

**Examples**

Some examples of collaborative filtering algorithms :
- YouTube content recommendation to users — recommending you videos based on other users who have subscribed / watched similar videos as yourself.
- CourseEra course recommendation — recommending you courses based on other individuals who have finished existing courses you’ve finished.

**Implementation**

- Import data from generate_data function (function provided above)
- Generate a pivot table with readers on the index and books on the column and values being the ratings
- Calculate similarity between items and users using svds
- Generate item recommendations based on user_id

## Functions

In [4]:
def normalize_pred_ratings(pred_ratings):
    '''
    This function will normalize the input pred_ratings
    
    params:
        pred_ratings (List -> List) : The prediction ratings 
    '''
    return (pred_ratings - pred_ratings.min()) / (pred_ratings.max() - pred_ratings.min())

In [5]:
def generate_prediction_df(mat, pt_df, n_factors):
    '''
    This function will calculate the single value decomposition of the input matrix
    given n_factors. It will then generate and normalize the user rating predictions.
    
    params:
        mat (CSR Matrix) : scipy csr matrix corresponding to the pivot table (pt_df)
        pt_df (DataFrame) : pandas dataframe which is a pivot table
        n_factors (Integer) : Number of singular values and vectors to compute. 
                              Must be 1 <= n_factors < min(mat.shape). 
    '''
    
    if not 1 <= n_factors < min(mat.shape):
        raise ValueError("Must be 1 <= n_factors < min(mat.shape)")
        
    # matrix factorization
    u, s, v = svds(mat, k = n_factors)
    s = np.diag(s)

    # calculate pred ratings
    pred_ratings = np.dot(np.dot(u, s), v) 
    pred_ratings = normalize_pred_ratings(pred_ratings)
    
    # convert to df
    pred_df = pd.DataFrame(
        pred_ratings,
        columns = pt_df.columns,
        index = list(pt_df.index)
    ).transpose()
    return pred_df

In [6]:
def recommend_items(pred_df, usr_id, n_recs):
    '''
    Given a usr_id and pred_df this function will recommend
    items to the user.
    
    params:
        pred_df (DataFrame) : generated from `generate_prediction_df` function
        usr_id (Integer) : The user you wish to get item recommendations for
        n_recs (Integer) : The number of recommendations you want for this user
    '''
    
    usr_pred = pred_df[usr_id].sort_values(ascending = False).reset_index().rename(columns = {usr_id : 'sim'})
    rec_df = usr_pred.sort_values(by = 'sim', ascending = False).head(n_recs)
    return rec_df

## Implementation

### Reading data

In [7]:
PATH = './data/data.csv'

In [8]:
data = pd.read_csv(PATH)
print(data.shape)


(100000, 10)


In [9]:
data.head()

,book_id,author_id,book_genre,reader_id,num_pages,book_rating,publisher_id,publish_year,book_price,text_lang
0,2671,204,8,17316,191,2,26,2002,149,7
1,2212,95,4,17605,659,8,29,2009,88,7
2,777,215,7,109,223,6,14,2001,116,4
3,372,275,1,18465,618,5,10,2011,125,4
4,1476,222,10,23128,619,5,4,2012,136,5


### Generating a pivot table with readers on the index and books on the column and values being the ratings

In [10]:
# Creating a copy
df = data.copy()

# Pivot table
pt_df = df.pivot_table(
    columns = 'book_id',
    index = 'reader_id',
    values = 'book_rating'
).fillna(0)

pt_df

book_id,1,2,3,4,5,6,7,8,9,10,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
reader_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Converting to a csr matrix

In [11]:
mat = pt_df.values
mat = csr_matrix(mat)
mat

<28935x3000 sparse matrix of type '<class 'numpy.float64'>'
	with 99935 stored elements in Compressed Sparse Row format>

In [12]:
pred_df = generate_prediction_df(mat, pt_df, 10)
pred_df

,1,2,3,4,5,6,7,8,9,10,...,29990,29991,29992,29993,29994,29995,29996,29997,29999,30000
book_id,,,,,,,,,,,,,,,,,,,,,
1,0.186107,0.185286,0.184746,0.184574,0.184377,0.184924,0.184996,0.185011,0.184712,0.184659,...,0.184594,0.184693,0.185372,0.184830,0.184825,0.185018,0.184659,0.185043,0.184896,0.184641
2,0.185679,0.185453,0.184876,0.184833,0.184952,0.185368,0.185257,0.185251,0.184967,0.184873,...,0.184515,0.184551,0.185241,0.184783,0.184749,0.185391,0.185128,0.184959,0.184927,0.184897
3,0.185296,0.184797,0.184694,0.184640,0.184839,0.184865,0.184820,0.184830,0.184681,0.184598,...,0.184519,0.185034,0.184863,0.184701,0.184670,0.184989,0.184817,0.184843,0.184739,0.184634
4,0.184792,0.184851,0.184636,0.184585,0.184655,0.185069,0.184520,0.184766,0.184746,0.184540,...,0.184573,0.184289,0.184697,0.184501,0.184672,0.184912,0.184397,0.184662,0.184694,0.184563
5,0.187632,0.186328,0.185168,0.184798,0.184636,0.186052,0.185727,0.185825,0.185137,0.185137,...,0.184553,0.185046,0.185478,0.185216,0.184949,0.186009,0.185143,0.185504,0.185165,0.184966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,0.185967,0.185179,0.184785,0.184772,0.186117,0.185048,0.185050,0.185120,0.184862,0.184974,...,0.184517,0.184586,0.185215,0.184783,0.184663,0.185311,0.184936,0.184861,0.184922,0.185027
2997,0.185413,0.185180,0.184731,0.184744,0.184887,0.184958,0.184985,0.185042,0.184751,0.184771,...,0.184546,0.184928,0.185279,0.184777,0.184841,0.185041,0.184978,0.185015,0.184941,0.184795
2998,0.184988,0.184676,0.184666,0.184838,0.185652,0.184840,0.184845,0.184959,0.184893,0.184587,...,0.184545,0.184933,0.185112,0.184607,0.184784,0.185153,0.184635,0.184838,0.184817,0.184748


### Generating recommendations

In [13]:
recommend_items(pred_df, usr_id=5, n_recs=5)

,book_id,sim
0,837,0.235284
1,404,0.200257
2,1345,0.194536
3,2015,0.193847
4,2679,0.192834


# Method 2. Content Based Systems

**Intuition**

Content based systems generates recommendations based on the users preferences and profile. They try to match users to items which they’ve liked previously. The level of similarity between items is generally established based on attributes of items liked by the user. Unlike most collaborative filtering models which leverage ratings between target user and other users, content based models focus on the ratings provided by the target user themselves. In essence, the content based approach leverages different sources of data to generate recommendations.

The simplest forms of content based systems require the following sources of data (these requirements can increase based on the complexity of the system you’re trying to build):

1. Item level data source — you need a strong source of data associated to the attributes of the item. For our scenario, we have things like book price, num_pages, published_year, etc. The more information you know regarding the item, the more beneficial it will be for your system.
2. User level data source — you need some sort of user feedback based on the item you’re providing recommendations for. This level of feedback can be either implicit or explicit. In our sample data, we’re working with user ratings of books they’ve read. The more user feedback you can track, the more beneficial it will be for your system.

**Advantages**

Content based models are most advantageous for recommending items when there is an insufficient amount of rating data available. This is because other items with similar attributes might have been rated by the user. Hence, a model should be able to leverage the ratings along with the item attributes to generate recommendations even when there isn’t a lot of data.

**Disadvantages**

There are two main disadvantages of content based systems.
1. The recommendations provided are “obvious” based on the items / content the user has consumed. This is a disadvantage because if the user has never interacted with a particular type of item, that item will never be recommended to the user. For example, if you’ve never read mystery books, then through this approach, you will never be recommended mystery books. This is because the model is user specific and doesn’t leverage knowledge from similar users. This reduces the diversity of the recommendations, this is a negative outcome for many businesses.
2. They’re ineffective for providing recommendations for new users. When building a model you require a history of explicit / implicit user level data for the items. It’s generally important to have a large dataset of ratings available to make robust predictions without overfitting.

**Examples**

Some examples of content based systems are :
- Amazon product feed (you’re being recommended products similar to what you’ve previously purchased)
- Spotify music recommendations

There are many excellent content based systems which are built algorithmically without the dependency on a model based approach. For example companies like Hacker Rank and Reddit have been known to previously used algorithmic approaches to recommend new posts on their platform to users. The key to building an algorithmic approach to content based recommenders lies in defining a set of rules for your business which can be used to rank items. In the case of Reddit, their recommendations are bounded by time of post, number of likes, number of dislikes, number of comments, etc. This can be factored into a formula to generate a score for a post, a high score would yield a high recommendation and vice versa.

**Implementation**

- Import data from generate_data function (function provided above)
- Normalize book_price, book_ratings, num_pages
- One hot encode publish_year, book_genre, text_lang
- Given a book_id input, calculate the cosine similarity and return top n books similar to the input

## Functions

In [14]:
def normalize_input_data(data):
    '''
    This function will normalize the input data to be between 0 and 1
    
    params:
        data (List) : The list of values you want to normalize
    
    returns:
        The input data normalized between 0 and 1
    '''
    min_val = min(data)
    if min_val < 0:
        data = [x + abs(min_val) for x in data]
    max_val = max(data)
    return [x/max_val for x in data]

In [15]:
def ohe(df, enc_col):
    '''
    This function will one hot encode the specified column and add it back
    onto the input dataframe
    
    params:
        df (DataFrame) : The dataframe you wish for the results to be appended to
        enc_col (String) : The column you want to OHE
    
    returns:
        The OHE columns added onto the input dataframe
    '''
    
    ohe_df = pd.get_dummies(df[enc_col])
    ohe_df.reset_index(drop = True, inplace = True)
    return pd.concat([df, ohe_df], axis = 1)

In [16]:
class CBRecommend():
    def __init__(self, df):
        self.df = df
        
    def cosine_sim(self, v1,v2):
        '''
        This function will calculate the cosine similarity between two vectors
        '''
        return sum(dot(v1,v2)/(norm(v1)*norm(v2)))
    
    def recommend(self, book_id, n_rec):
        """
        df (dataframe): The dataframe
        song_id (string): Representing the song name
        n_rec (int): amount of rec user wants
        """
        
        # calculate similarity of input book_id vector w.r.t all other vectors
        inputVec = self.df.loc[book_id].values
        self.df['similarity']= self.df.apply(lambda x: self.cosine_sim(inputVec, x.values), axis=1)

        # returns top n user specified books
        return self.df.nlargest(columns='similarity',n=n_rec)

## Implementation

### Reading data

In [17]:
PATH = './data/data.csv'

In [18]:
data = pd.read_csv(PATH)
print(data.shape)


(100000, 10)


In [19]:
data.head()

,book_id,author_id,book_genre,reader_id,num_pages,book_rating,publisher_id,publish_year,book_price,text_lang
0,2671,204,8,17316,191,2,26,2002,149,7
1,2212,95,4,17605,659,8,29,2009,88,7
2,777,215,7,109,223,6,14,2001,116,4
3,372,275,1,18465,618,5,10,2011,125,4
4,1476,222,10,23128,619,5,4,2012,136,5


### Transformations

In [20]:
# Creating a copy
df = data.copy()

# Normalizing the num_pages, ratings, price columns
df['num_pages_norm'] = normalize_input_data(df['num_pages'].values)
df['book_rating_norm'] = normalize_input_data(df['book_rating'].values)
df['book_price_norm'] = normalize_input_data(df['book_price'].values)

# OHE on publish_year and genre
df = ohe(df = df, enc_col = 'publish_year')
df = ohe(df = df, enc_col = 'book_genre')
df = ohe(df = df, enc_col = 'text_lang')

# Drop redundant columns
cols = ['publish_year', 'book_genre', 'num_pages', 'book_rating', 'book_price', 'text_lang']
df.drop(columns = cols, inplace = True)
df.set_index('book_id', inplace = True)

df.head()

,author_id,reader_id,publisher_id,num_pages_norm,book_rating_norm,book_price_norm,2000,2001,2002,2003,...,8,9,10,1,2,3,4,5,6,7
book_id,,,,,,,,,,,,,,,,,,,,,
2671,204,17316,26,0.272857,0.2,0.745,0,0,1,0,...,1,0,0,0,0,0,0,0,0,1
2212,95,17605,29,0.941429,0.8,0.440,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
777,215,109,14,0.318571,0.6,0.580,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
372,275,18465,10,0.882857,0.5,0.625,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1476,222,23128,4,0.884286,0.5,0.680,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0


### Run on a sample as an example

In [21]:
t = df.copy()
cbr = CBRecommend(df = t)
cbr

In [22]:
cbr.recommend(book_id = 1042, n_rec = 5)[['author_id', 'book_rating_norm', 'similarity']].sort_values(by=['book_rating_norm'], ascending = False)

,author_id,book_rating_norm,similarity
book_id,,,
1046,392,1.0,5.08079
1569,362,0.9,5.08079
1108,432,0.7,5.08079
1362,423,0.5,5.08079
514,435,0.1,5.08079


# Method 3. Hybrid Recommendation System

**Intuition**

Various methods of recommendations systems have their own benefits and flaws. Often, many of these methods may seem restrictive when used in isolation, especially when multiple sources of data is available for the problem. Hybrid recommender systems are ones designed to use different available data sources to generate robust inferences.

Hybrid recommendation systems have two predominant designs, parallel and sequential. The parallel design provides the input to multiple recommendation systems, each of those recommendations are combined to generate one output. The sequential design provides the input parameters to a single recommendation engine, the output is passed on to the following recommender in a sequence.

**Advantages**

Hybrid systems combine different models to combat the disadvantages of one model with another. This overall reduces the weaknesses of using individual models and aids in generating more robust recommendations. This yields more robust and personalized recommendations for users.

**Disadvantages**

These types of models generally have high computational complexity and require a large database of ratings and other attributes to keep up to date. Without up to date metrics (user engagement, ratings, etc.) it makes it difficult to retrain and provide new recommendations with updated items and ratings from various users.

**Examples**

Netflix is a company which uses a hybrid recommendation system, they generate recommendations to users based on the watch and search style of similar users (collaborative filtering) in conjunction with movies which share similar characteristics who’ve been rated by users (content based).

**Implementation**

- Import data from generate_data function (function provided above)
- Use a content-based model (cosine_similarity) to compute the 50 most similar books
- Compute the predicted ratings that the user might give these 50 books using a collaborative filtering model (SVD)
- Return the top n books with the highest predicted rating

## Functions

In [23]:
def hybrid(reader_id, book_id, n_recs, df, cosine_sim, svd_model):
    '''
    This function represents a hybrid recommendation system, it will have the following flow:
        1. Use a content-based model (cosine_similarity) to compute the 50 most similar books
        2. Compute the predicted ratings that the user might give these 50 books using a collaborative filtering model (SVD)
        3. Return the top n books with the highest predicted rating
        
    params:
        reader_id (Integer) : The reader_id 
        book_id (Integer) : The book_id 
        n_recs (Integer) : The number of recommendations you want
        df (DataFrame) : Original dataframe with all book information 
        cosine_sim (DataFrame) : The cosine similarity dataframe
        svd_model (Model) : SVD model
    '''
    
    # sort similarity values in decreasing order and take top 50 results
    similarity = list(enumerate(cosine_sim[int(book_id)]))
    similarity = sorted(similarity, key=lambda x: x[1], reverse=True)
    similarity = similarity[1:50]
    
    # get book metadata
    book_idx = [i[0] for i in similarity]
    books = df.iloc[book_idx][['book_id', 'book_rating', 'num_pages', 'publish_year', 'book_price', 'reader_id']]
    
    # predict using the svd_model
    books['est'] = books.apply(lambda x: svd_model.predict(reader_id, x['book_id'], x['book_rating']).est, axis = 1)
    
    # sort predictions in decreasing order and return top n_recs
    books = books.sort_values('est', ascending=False)
    return books.head(n_recs)

## Implementation

### Reading data

In [24]:
PATH = './data/data.csv'

In [25]:
data = pd.read_csv(PATH)
print(data.shape)


(100000, 10)


In [26]:
data.head()

,book_id,author_id,book_genre,reader_id,num_pages,book_rating,publisher_id,publish_year,book_price,text_lang
0,2671,204,8,17316,191,2,26,2002,149,7
1,2212,95,4,17605,659,8,29,2009,88,7
2,777,215,7,109,223,6,14,2001,116,4
3,372,275,1,18465,618,5,10,2011,125,4
4,1476,222,10,23128,619,5,4,2012,136,5


### Content based

In [27]:
# Creating a copy
df = data.copy()

# Pivot table
rmat = df.pivot_table(
    columns = 'book_id',
    index = 'reader_id',
    values = 'book_rating'
).fillna(0)

### Compute the cosine similarity matrix 

In [28]:
cosine_sim = cosine_similarity(rmat, rmat)
cosine_sim = pd.DataFrame(cosine_sim, index=rmat.index, columns=rmat.index)
cosine_sim

reader_id,1,2,3,4,5,6,7,8,9,10,...,29990,29991,29992,29993,29994,29995,29996,29997,29999,30000
reader_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
29996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
29997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


### Collaborative filtering

In [29]:
reader = Reader()
data = Dataset.load_from_df(df[['reader_id', 'book_id', 'book_rating']], reader)
data

### Modeling

In [30]:
# split data into train test
trainset, testset = train_test_split(data, test_size=0.3,random_state=10)

# train model
svd = SVD()
svd.fit(trainset)

In [31]:
# run the trained model against the testset
test_pred = svd.test(testset)

# get RMSE
accuracy.rmse(test_pred, verbose=True)

RMSE: 2.9407


2.9407288315417364

### Generate recommendations

In [32]:
hybrid(reader_id = df['reader_id'].values[0], 
       book_id = df['book_id'].values[0], 
       n_recs = 10, 
       df=df, 
       cosine_sim=cosine_sim, 
       svd_model=svd
      ).sort_values(by=['book_rating'], ascending = False)

,book_id,book_rating,num_pages,publish_year,book_price,reader_id,est
15087,1613,10,688,2020,141,4532,5.0
22653,2449,8,435,2015,40,2341,5.0
18189,2574,7,195,2009,133,4064,5.0
13251,1832,6,675,2006,181,3513,5.0
3782,2434,5,501,2013,133,16436,5.0
23509,1723,5,604,2017,145,20288,5.0
2815,1144,4,478,2014,1,16712,5.0
28382,2901,3,637,2003,171,11812,5.0
16295,2627,3,342,2000,56,16636,5.0
24115,2139,1,136,2019,22,27983,5.0


In [33]:
df.head()

,book_id,author_id,book_genre,reader_id,num_pages,book_rating,publisher_id,publish_year,book_price,text_lang
0,2671,204,8,17316,191,2,26,2002,149,7
1,2212,95,4,17605,659,8,29,2009,88,7
2,777,215,7,109,223,6,14,2001,116,4
3,372,275,1,18465,618,5,10,2011,125,4
4,1476,222,10,23128,619,5,4,2012,136,5


# Method 4. fpGrowth - Pyspark using different type of data (associating different items bought in different purchases by customers)

**Intuition**

#TODO

## Implementation

### fpGrwowth

In [67]:
spark = SparkSession\
  .builder\
  .appName("test_import")\
  .enableHiveSupport()\
  .getOrCreate()

In [68]:
def generate_data_spark(n_transactions, n_items, dataset_size):
    '''
    This function will generate a dataset with features associated to
    items purchased in a specific transaction. : 
        - transaction_id (String) : Unique identified for the book
        - purchased_item_id (String) : Unique identifier for the user
                
    params:
        n_transactions (Integer) : The number of transactions you want the dataset to have
        n_items (Integer) : Number of items to be chosen from
        dataset_size (Integer) : The number of rows to be generated 
        
    example:
        data = generate_data_spark()
    '''
    
    df = pd.DataFrame(
        {
            'transaction_id' : [randint(1, n_transactions) for _ in range(dataset_size)],
            'purchased_item_id': [sorted(random.sample(range(1, n_items), randint(1, 15))) for _ in range(dataset_size)],  # 15 is maximum number of purchased items per customer
            #'purchased_item_id' : [list(np.random.randint(n_items, size = randint(1, 15))) for _ in range(dataset_size)],   # Using numpy. It has some problems with spark
        }
    ).drop_duplicates(subset='transaction_id', keep='first')
    
    # Creating schema for converting to spark df
    mySchema = StructType([
        StructField('transaction_id', LongType(), True),
        StructField('purchased_item_id', ArrayType(LongType()),True),  
    ])
    sparkDF=spark.createDataFrame(df, schema = mySchema) 
    return sparkDF

In [69]:
df = generate_data_spark(n_transactions = 1200, n_items = 20, dataset_size = 10000)
df.head(20)
df

DataFrame[transaction_id: bigint, purchased_item_id: array<bigint>]

In [70]:
df.printSchema()

root
 |-- transaction_id: long (nullable = true)
 |-- purchased_item_id: array (nullable = true)
 |    |-- element: long (containsNull = true)



In [71]:
df.show(truncate=False)

+--------------+-----------------------------------------------------+
|transaction_id|purchased_item_id                                    |
+--------------+-----------------------------------------------------+
|409           |[10]                                                 |
|429           |[1, 8, 9, 10, 11, 15, 17, 19]                        |
|324           |[1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 18, 19]  |
|245           |[1, 3, 4, 5, 7, 9, 10, 12, 13, 14, 15, 19]           |
|608           |[2, 4, 6, 7, 8, 11, 14, 15, 17, 18, 19]              |
|468           |[1, 2, 3, 6, 7, 8, 10, 12, 13, 15, 17, 18]           |
|697           |[1, 2, 4, 7, 8, 9, 10, 12, 13, 14, 16, 17, 18]       |
|59            |[2, 4, 5, 6, 7, 8, 9, 11, 14, 15, 16, 17, 18, 19]    |
|960           |[12]                                                 |
|973           |[3, 4, 7, 10, 13, 14, 15, 16, 17, 18]                |
|916           |[3, 4, 5, 7, 9, 11, 12, 13, 14, 17, 18]              |
|309  

In [72]:
fp = FPGrowth(itemsCol='purchased_item_id', minSupport=0.1, minConfidence=0.6)
model = fp.fit(df)
model

FPGrowthModel: uid=FPGrowth_8d5761543326, numTrainingRecords=1200

In [73]:
# Display frequent itemsets.
model.freqItemsets.show(10)

+-----------+----+
|      items|freq|
+-----------+----+
|       [17]| 490|
|    [17, 1]| 254|
|[17, 1, 11]| 148|
|[17, 1, 13]| 152|
| [17, 1, 7]| 153|
| [17, 1, 5]| 150|
|[17, 1, 18]| 144|
|[17, 1, 19]| 153|
| [17, 1, 6]| 134|
| [17, 1, 3]| 147|
+-----------+----+
only showing top 10 rows



In [74]:
# Display generated association rules.
model.associationRules.show(10)

+----------+----------+------------------+------------------+-------------------+
|antecedent|consequent|        confidence|              lift|            support|
+----------+----------+------------------+------------------+-------------------+
|   [1, 13]|      [19]|0.6072727272727273| 1.406809406809407|0.13916666666666666|
|   [1, 13]|       [3]|0.6036363636363636|1.3641499743194658|0.13833333333333334|
|   [1, 13]|       [4]|               0.6|1.4285714285714286|             0.1375|
|  [15, 14]|      [19]|0.6062992125984252|1.4045541604596725|0.12833333333333333|
|  [16, 11]|       [5]|0.6075471698113207|1.4379814670090432|0.13416666666666666|
|  [16, 11]|       [8]|0.6075471698113207|1.4552028019432832|0.13416666666666666|
|  [16, 14]|       [1]|0.6037735849056604|1.4375561545372866|0.13333333333333333|
|   [17, 3]|      [19]|               0.6|  1.38996138996139|              0.135|
|   [10, 6]|      [14]| 0.601593625498008|1.4323657749952572|0.12583333333333332|
|   [10, 6]|    

In [75]:
# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(df).show(10)

+--------------+--------------------+--------------------+
|transaction_id|   purchased_item_id|          prediction|
+--------------+--------------------+--------------------+
|           409|                [10]|                  []|
|           429|[1, 8, 9, 10, 11,...|[13, 18, 3, 16, 5...|
|           324|[1, 2, 3, 4, 5, 6...|    [10, 16, 15, 17]|
|           245|[1, 3, 4, 5, 7, 9...|[18, 16, 6, 2, 11...|
|           608|[2, 4, 6, 7, 8, 1...|[9, 16, 13, 3, 1,...|
|           468|[1, 2, 3, 6, 7, 8...|[19, 4, 14, 9, 11...|
|           697|[1, 2, 4, 7, 8, 9...|[19, 3, 11, 5, 15...|
|            59|[2, 4, 5, 6, 7, 8...|  [1, 13, 3, 12, 10]|
|           960|                [12]|                  []|
|           973|[3, 4, 7, 10, 13,...|[19, 1, 11, 9, 5,...|
+--------------+--------------------+--------------------+
only showing top 10 rows

